# Gradient Descent Step Sizes Lab

In [2]:
first_show = {'budget': 100, 'revenue': 275}
second_show = {'budget': 200, 'revenue': 300}
third_show = {'budget': 400, 'revenue': 700}

shows = [first_show, second_show, third_show]

In [3]:
from linear_equations import build_regression_line

budgets = list(map(lambda show: show['budget'], shows))
revenues = list(map(lambda show: show['revenue'], shows))

In [3]:
#def regression_line(x):
#    return .6*x + 133.33

In [4]:
def squared_error(x, y, m, b):
    return (y - (m*x + b))**2

def residual_sum_squares(x_values, y_values, m, b):
    data_points = list(zip(x_values, y_values))
    squared_errors = map(lambda data_point: squared_error(data_point[0], data_point[1], m, b), data_points)
    return sum(squared_errors)

In [5]:
#from plotly.offline import iplot, init_notebook_mode
#from graph import plot, m_b_trace
#import plotly.graph_objs as go

#init_notebook_mode(connected=True)

#from graph import trace_values, plot

#data_trace = trace_values(budgets, revenues)
#regression_trace = m_b_trace(.6, 133.33, budgets)
#plot([data_trace, regression_trace])

### Building a cost curve

Now let's use the `residual_sum_squares` function to build a cost curve.  Keeping the $b$ value fixed at $133.33$, write a function called `rss_values`.  
* `rss_values` passes our dataset with the `x_values` and `y_values` arguments.  
* It also takes a list of values of $m$, and an initial $b$ value as arguments.  
* It outputs a dictionary with keys of `m_values` and `rss_values`, with each key pointing to a list of the corresponding values.

In [6]:
# goal: build cost curve m vs e
#step 1: outputs dict with keys m_values, rss_values; each key points to list of corresponding values

def rss_values(x_values, y_values, m_values, b):
    z={'m_values':m_values}
    w=[]
    for m in m_values:
        k=residual_sum_squares(x_values, y_values, m, b)
        w.append(k)
    z.update({'rss_values':w})
    return z  

In [7]:
x_values = list(map(lambda show: show['budget'] ,shows))
y_values = list(map(lambda show: show['revenue'] ,shows))
initial_m_values = list(range(8, 19, 1))
scaled_m_values = list(map(lambda initial_m_value: initial_m_value/10,initial_m_values))
b_value = 133.33
rss_values(x_values, y_values, scaled_m_values, b_value)

# {'m_values': [0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8],
#  'rss_values': [64693.76669999998,
#   45559.96669999998,
#   30626.166699999987,
#   19892.36669999999,
#   13358.5667,
#   11024.766700000004,
#   12890.96670000001,
#   18957.166700000016,
#   29223.36670000002,
#   43689.566700000025,
#   62355.76670000004]}


{'m_values': [0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8],
 'rss_values': [64693.76669999998,
  45559.96669999998,
  30626.166699999987,
  19892.36669999999,
  13358.5667,
  11024.766700000004,
  12890.96670000001,
  18957.166700000016,
  29223.36670000002,
  43689.566700000025,
  62355.76670000004]}

In [8]:
#z={'a':[1,2,3],'b':[12,9,7]}
#z.values()

In [18]:
#sidenote: plotting a table

from plotly.offline import iplot, init_notebook_mode
from graph import plot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

def plot_table(headers, columns):
    plot([go.Table(header=dict(values=['M values', 'RSS values'], cells=dict(values=columns)))])

In [19]:
cost_chart = rss_values(budgets, revenues, scaled_m_values, b_value) 
column_values = list(cost_chart.values())
plot_table(headers = ['M values', 'RSS values'], columns=column_values)

ValueError: Invalid property specified for object of type plotly.graph_objs.table.Header: 'cells'

    Valid properties:
        align
            Sets the horizontal alignment of the `text` within the
            box. Has an effect only if `text` spans more two or
            more lines (i.e. `text` contains one or more <br> HTML
            tags) or if an explicit width is set to override the
            text width.
        alignsrc
            Sets the source reference on plot.ly for  align .
        fill
            plotly.graph_objects.table.header.Fill instance or dict
            with compatible properties
        font
            plotly.graph_objects.table.header.Font instance or dict
            with compatible properties
        format
            Sets the cell value formatting rule using d3 formatting
            mini-language which is similar to those of Python. See
            https://github.com/d3/d3-3.x-api-
            reference/blob/master/Formatting.md#d3_format
     

And let's plot this out using a a line chart.

In [11]:
#plotting same contents as table with a line chart

from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
from graph import plot, trace_values

initial_m_values = list(range(1, 18, 1))
scaled_m_values = list(map(lambda initial_m_value: initial_m_value/10,initial_m_values))
cost_values = rss_values(budgets, revenues, scaled_m_values, 133.33)

if cost_values:
    rss_trace = trace_values(cost_values['m_values'], cost_values['rss_values'], mode = 'lines')
    plot([rss_trace])

### Looking at the slope of our cost curve

In this section, we'll work up to building a gradient descent function that automatically changes our step size.  To get you started, we'll provide a function called `slope_at` that calculates the slope of the cost curve at a given point on the cost curve.  Here it is in action:

In [17]:
from helper import slope_at

In [18]:
slope_at(budgets, revenues, .6, 133.33333333333326)

{'m': 0.6, 'slope': -296312.3333330441}

In [19]:
slope_at(budgets, revenues, 1.6, 133.33333333333326)

{'m': 1.6, 'slope': 123687.66666691954}

So the `slope_at` function takes in our dataset, and returns the slope of the cost curve at that point.  So the numbers -296312 and 123877 reflect the slopes at the cost curve when m is .6 and 1.6 respectively.

![](./tangent-lines.png)

As you can see, it seems pretty accurate.  When the curve is steeper and downwards at $m = 6$, the slope is around -290,000.  And at $m = 1.3$ with our cost curve pointing upwards yet flatter, our slope is around 120,000. 

### Moving towards gradient descent

Now that we are familiar with our `slope_at` function and how it calculates the slope of our cost curve at a given point, we can begin to use that function with our gradient descent procedure.

Remember that gradient descent works by starting at a regression line with values m, and b, which corresponds to a point on our cost curve.  Then we alter our m or b value (here, the m value) by looking to the slope of the cost curve at that point.  Then we look to the slope of the cost curve at the new m value to indicate the size and direction of the next step.

So now let's write a function called `updated_m`.  The function will tell us the step size and direction to move along our cost curve.  The `updated_m` function takes as arguments an initial value of $m$, a learning rate, and the `slope` of the cost curve at that value of $m$.  Its return value is the next value of `m` that it calculates.

In [41]:
from error import residual_sum_squares

def updated_m(m_i0, learning_rate, cost_curve_slope):
    return m_i0 - learning_rate*cost_curve_slope

This is what our function returns.

In [44]:
current_slope = slope_at(budgets, revenues, 1.7, 133.33333333333326)['slope']
current_slope
updated_m(1.7, .000001, current_slope)
# 1.5343123333335096

current_slope = slope_at(budgets, revenues, 1.534, 133.33333333333326)['slope']
updated_m(1.534, .000001, current_slope)
# 1.43803233333338

current_slope = slope_at(budgets, revenues, 1.438, 133.33333333333326)['slope']
updated_m(1.438, .000001, current_slope)
# 1.3823523333332086

1.3823523333332086

Take a careful look at how we use the `updated_m` function.  By using our updated value of $m$ we are quickly converging towards an optimal value of $m$.   

Now let's write another function called `gradient_descent`.  The inputs of the function are `x_values`, `y_values`, `steps`, the `b` we are holding constant, the `learning_rate`, and the `current_m` that we are looking at.  The `steps` arguments represents the number of steps the function will take before the function stops.  We can get a sense of the return value in the cell below.  It is a list of dictionaries, with each dictionary having a key of the current `m` value, the `slope` of the cost curve at that m value, and the `rss` at that m value.

In [ ]:
#def updated_m(m_i0, learning_rate, cost_curve_slope):
  #  return m_i0 - learning_rate*cost_curve_slope

In [ ]:
#slope_at(budgets, revenues, .6, 133.33333333333326)

In [47]:
def gradient_descent(x_values, y_values, steps, b, learning_rate, current_m):
    z=[]
    for s in list(range(0,steps)):
        current_slope = slope_at(x_values, y_values, current_m, b)['slope']
        k={'m':current_m,'rss':residual_sum_squares(x_values, y_values, current_m, b), 'slope':current_slope}
        z.append(k)
        current_m = updated_m(current_m, learning_rate, current_slope)
    return z  

In [60]:
descent_steps = gradient_descent(budgets, revenues, 12, 133.33, learning_rate = .000001, current_m = 0)
#no problem
# [{'m': 0, 'rss': 368964.16669999994, 'slope': -548316.9999998063},
#  {'m': 0.5483169999998062, 'rss': 131437.9413767516, 'slope': -318023.86000024853},
#  {'m': 0.8663408600000547,  'rss': 51531.31420747324, 'slope': -184453.83880040026}, 
#  {'m': 1.050794698800455,  'rss': 24649.097944855268,  'slope': -106983.22650372575},
#  {'m': 1.1577779253041809,  'rss': 15604.976802103287,  'slope': -62050.271372208954},
#  {'m': 1.2198281966763898,  'rss': 12561.987166284125,  'slope': -35989.15739588847},
#  {'m': 1.2558173540722781,  'rss': 11538.008028425651, 'slope': -20873.711289696075},
#  {'m': 1.2766910653619743,  'rss': 11193.357340315255,  'slope': -12106.752547970245},
#  {'m': 1.2887978179099446,  'rss': 11077.310067278091,  'slope': -7021.916477824561},
#  {'m': 1.295819734387769,  'rss': 11038.209831325046,  'slope': -4072.711557128059},
#  {'m': 1.299892445944897,  'rss': 11025.020590634533,  'slope': -2362.172703124088},
#  {'m': 1.302254618648021,  'rss': 11020.562895703,  'slope': -1370.0601677373925}]

In [61]:
if descent_steps:
    m_values = list(map(lambda step: step['m'],descent_steps))
    rss_result_values = list(map(lambda step: step['rss'], descent_steps))
    text_values = list(map(lambda step: 'cost curve slope: ' + str(step['slope']), descent_steps))
    gradient_trace = trace_values(m_values, rss_result_values, text=text_values)
    plot([gradient_trace])

In [62]:
from plotly.offline import iplot, init_notebook_mode
from graph import plot, m_b_trace
import plotly.graph_objs as go

init_notebook_mode(connected=True)

from graph import trace_values, plot

data_trace = trace_values(budgets, revenues)
if descent_steps:
    m_values = list(map(lambda step: step['m'] ,descent_steps))
    regression_traces = list(map(lambda m: m_b_trace(m, 133.33, budgets, name = 'm:' + str(round(m, 2))), m_values))
    plot([data_trace, *regression_traces])

As you can see the slope converges towards a slope that better matches our data, around m = 1.3.  You can isolate how the line changes clicking on the names of the lines to the right, which toggles the display of the respective lines.